In [10]:
import numpy as np
import matplotlib
from mido import Message, MetaMessage
import mido
import sys
import random
import os
import time
import math
pwd = "/home/zedroman/Documents/Sonia_Data/Songs/anime_samples2/"
savepwd="/home/zedroman/Documents/Sonia_Data/Songs/_test2/"
music = sorted([f for f in os.listdir(pwd) if f[len(f)-4:len(f)] == ".mid"])


tempo = 500000 * 5
hidden_size = 512
max_len = 100

velocity_range = [0, 64]
time_range = [0, 150, 300];
tempo = 2000000 // 3
pedal_max = 20
max_len = 500
use_pedal = False

decoder_params = [velocity_range, time_range, tempo, pedal_max]

In [11]:
music = music[0:10]
print(music)

['0.mid', '1.mid', '10.mid', '11.mid', '2.mid', '4.mid', '5.mid', '6.mid', '7.mid', '8.mid']


In [12]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
device="cpu"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [13]:
def create_random(music):
    randoms = []
    for i in range(len(music)):
        r = []
        for j in range(3):
            r.append([random.random() * 10 for i in range(hidden_size)])
        randoms.append(r)
    return randoms
random_starts = create_random(music)

In [14]:
use_notes = {}
class_note = {}
for file in (music):
    midi_file = mido.MidiFile(pwd + file)
    for track in midi_file.tracks:
        for msg in track:
            if (msg.type == "note_on"):
                if (msg.note not in use_notes):
                    class_note[len(use_notes)] = msg.note
                    use_notes[msg.note] = len(use_notes)
classes = len(use_notes) * len(time_range) * len(velocity_range)
sos = classes
classes += 1
print(use_notes)
print(classes)
decoder_params.append(class_note)

{61: 0, 66: 1, 68: 2, 69: 3, 38: 4, 50: 5, 65: 6, 59: 7, 37: 8, 49: 9, 57: 10, 42: 11, 30: 12, 62: 13, 35: 14, 47: 15, 71: 16, 40: 17, 52: 18, 64: 19, 33: 20, 45: 21, 73: 22, 56: 23, 54: 24, 32: 25, 44: 26, 76: 27, 74: 28, 75: 29, 39: 30, 77: 31, 78: 32, 51: 33, 79: 34, 72: 35, 70: 36, 67: 37, 36: 38, 48: 39, 34: 40, 46: 41, 82: 42, 63: 43, 84: 44, 60: 45, 41: 46, 53: 47, 80: 48, 58: 49, 55: 50, 43: 51, 27: 52, 29: 53, 31: 54, 81: 55, 83: 56, 86: 57, 28: 58, 26: 59, 88: 60, 85: 61, 95: 62, 90: 63, 91: 64, 96: 65, 93: 66, 87: 67}
409


In [15]:
msg = Message("note_on", note = class_note[0], velocity = 0, time= time_range[1])
print(class_note[0])
print(msg_to_class(msg))
print(class_to_msg(msg_to_class(msg)))

61


NameError: name 'msg_to_class' is not defined

In [16]:
def list_find(l, n):
    for i in range(len(l)):
        if l[i] == n:
            return i
    return -1

def msg_to_class(msg):
    note = use_notes[msg.note]
    t = list_find(time_range, msg.time)
    v = list_find(velocity_range, msg.velocity)
    return v * len(use_notes) * len(time_range) + t * len(use_notes) + note
    
def class_to_msg(n):
    note = class_note[n % len(use_notes)]
    n //= len(use_notes)
    t = time_range[n % len(time_range)]
    n //= len(time_range)
    v = velocity_range[n]
    return Message("note_on", note = note, velocity = v, time= t)


def midi_to_list(mid=None, pwd=None, tensor=True):
    if(mid==None):
          mid = mido.MidiFile(pwd)
    sample = []
    for i, track in enumerate(mid.tracks):
        for msg in track:
            if (msg.type == "note_on"):
                sample.append(msg_to_class(msg))
    if (tensor):
        return torch.tensor(sample, device=device)
    else:
        return sample

def list_to_midi(sample):
    midi = mido.MidiFile(type=0)
    tr = mido.MidiTrack()
    midi.tracks.append(tr)
    tr.append(MetaMessage("set_tempo", tempo=tempo, time=0))
    
    tr.append(MetaMessage("key_signature", key='A', time=0))
    
    pedal = pedal_max
    last_notes = []
    
    for i in sample:
        if (use_pedal):
            if (pedal == 0):
                for j in range(len(last_notes) - 1):
                    tr.append(Message("note_off", note=last_notes[j], velocity=0, time=0, channel=0))
                pedal = pedal_max
                last_notes = [last_notes[-1]]
        tr.append(class_to_msg(i))
        if (use_pedal):
            pedal -= 1
            last_notes.append(class_note[i % len(use_notes)])
    if (use_pedal):
        for i in last_notes:
            tr.append(Message("note_off", note=i, velocity=0, time=0, channel=0))
    tr.append(MetaMessage("end_of_track", time = 0))
    return midi

In [17]:

for file in (music):
    midi_file = mido.MidiFile(pwd + file)
    sample = midi_to_list(mid=midi_file, tensor=False)
    print(file)
    midi = list_to_midi(sample)
    midi.save("/home/zedroman/Documents/Sonia_Data/Songs/_test2/test_to.mid")
    break
    

0.mid


In [18]:

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [19]:

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, other):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.gru1 = nn.GRU(classes, hidden_size)
        self.gru2 = nn.GRU(hidden_size, hidden_size)
        self.gru3 = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

        self.velocity = other[0]
        self.time_range = other[1]
        self.tempo = other[2]
        self.pedal_max = other[3]
        self.notes = other[4]

    def forward(self, input, hidden):
        output, hidden[0] = self.gru1(input, hidden[0])
        output = F.relu(output)
        output, hidden[1] = self.gru2(output, hidden[1])
        output = F.relu(output)
        output, hidden[2] = self.gru3(output, hidden[2])
        output = self.softmax(self.out(output[0]))
        
        return output, hidden

In [20]:

def one_hot(c):
    x = torch.zeros(classes, device=device, dtype=torch.float)
    x[c] = 1;
    return x.view(1,1,-1)

def train(target_tensor, decoder, decoder_optimizer, criterion, max_length=max_len, num = -1):
    decoder_optimizer.zero_grad()
    target_length = target_tensor.size(0)
    loss = 0
    
    decoder_input = one_hot(sos)
    decoder_hidden = [torch.tensor(random_starts[num][0], device=device, dtype=torch.float).view(1,1,-1),
                        torch.tensor(random_starts[num][1], device=device, dtype=torch.float).view(1,1,-1),
                        torch.tensor(random_starts[num][2], device=device, dtype=torch.float).view(1,1,-1)]

    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(
            decoder_input, decoder_hidden)
        loss += criterion(decoder_output, target_tensor[di:di+1])
        decoder_input = one_hot(target_tensor[di])  # Teacher forcing

    loss.backward()

    decoder_optimizer.step()

    return loss.item() / target_length

In [21]:
def trainIters(decoder, n_iters,decoder_optimizer, print_every=len(music), epoch=0, save_mod=False):
    start = time.time()
    print_loss_total = 0  # Reset every print_every
    criterion = nn.NLLLoss()
    save = 20
    for iter in range(1, n_iters + 1):
        training_pair = music[iter % (len(music))]
        midi_file = mido.MidiFile(pwd + training_pair)
        target_tensor = midi_to_list(mid=midi_file)
        loss = train(target_tensor,
                     decoder, decoder_optimizer, criterion, num = iter % (len(music)))
        print_loss_total += loss

        
        if iter % print_every == 0:
            save -= 1
            last_loss = print_loss_total / print_every
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))
            if (save == 0 and save_mod):
                save = 20
                torch.save({
                    'model_state_dict': decoder.state_dict(),
                    'optimizer_state_dict': decoder_optimizer.state_dict(),
                    'random_starts' : random_starts,
                    'velocity_range' : velocity_range,
                    'time_range' : time_range,
                    'tempo' : tempo,
                    'pedal_max' : pedal_max,
                    'class_note' : class_note,
                    }, '/home/zedroman/Documents/Sonia/models/bethoven2')
                with open('/home/zedroman/Documents/Sonia/models/bethoven2_classes', 'w') as f: 
                    f.write(str(classes))
                    f.write(' ')
                    f.write(str(hidden_size))

In [22]:
learning_rate = 0.001
decoder = DecoderRNN(hidden_size, classes, decoder_params).to(device)
decoder_optimizer = optim.Adam(decoder.parameters() , lr=learning_rate)

In [37]:
trainIters(decoder, 2000, decoder_optimizer,save_mod=False)

0m 59s (- 198m 47s) (10 0%) 0.0143
1m 58s (- 195m 39s) (20 1%) 0.0141
2m 57s (- 194m 16s) (30 1%) 0.0138
3m 55s (- 192m 25s) (40 2%) 0.0142
4m 53s (- 190m 38s) (50 2%) 0.0141
5m 52s (- 189m 56s) (60 3%) 0.0139
6m 49s (- 188m 15s) (70 3%) 0.0139
7m 49s (- 187m 39s) (80 4%) 0.0137
8m 48s (- 186m 47s) (90 4%) 0.0131
9m 47s (- 186m 6s) (100 5%) 0.0128
10m 46s (- 185m 4s) (110 5%) 0.0129
11m 45s (- 184m 20s) (120 6%) 0.0129
12m 47s (- 183m 59s) (130 6%) 0.0125
13m 45s (- 182m 46s) (140 7%) 0.0124
14m 43s (- 181m 42s) (150 7%) 0.0121
15m 41s (- 180m 23s) (160 8%) 0.0115
16m 38s (- 179m 6s) (170 8%) 0.0112
17m 35s (- 177m 51s) (180 9%) 0.0109
18m 33s (- 176m 46s) (190 9%) 0.0107
19m 32s (- 175m 51s) (200 10%) 0.0105
20m 32s (- 175m 7s) (210 10%) 0.0104
21m 30s (- 174m 1s) (220 11%) 0.0103
22m 27s (- 172m 51s) (230 11%) 0.0103
23m 24s (- 171m 42s) (240 12%) 0.0101
24m 22s (- 170m 34s) (250 12%) 0.0100
25m 18s (- 169m 25s) (260 13%) 0.0097
26m 17s (- 168m 24s) (270 13%) 0.0097
27m 16s (- 167m 3

KeyboardInterrupt: 

In [27]:
def evaluate(decoder,num=-1, max_length=max_len):
    with torch.no_grad():

        decoder_input = one_hot(sos)
        decoder_hidden = []
        for j in range(3):
            decoder_hidden.append([random.random() * 10 for i in range(hidden_size)])
        decoder_hidden =   [torch.tensor(decoder_hidden[0], device=device).view(1,1,-1),
                        torch.tensor(decoder_hidden[1], device=device).view(1,1,-1),
                        torch.tensor(decoder_hidden[2], device=device).view(1,1,-1)]

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(random.randint(1,2))
            topv = topv.max().item() - topv.cpu()[0].numpy()
            summ = topv.sum()
            chosen = np.random.choice(topi[0].cpu(), p= topv / summ)
            decoder_input = one_hot(chosen)
            decoded_words.append(chosen)

        return decoded_words
    
def generate(number, decoder):
    for i in range(number):
        new_music = evaluate(decoder, max_length=1000)
        midi = list_to_midi(new_music)
        print(midi.length)
        midi.save(savepwd +"test" + str(i) + ".mid")
    return midi
use_pedal=False
last = generate(10, decoder)
for track in last.tracks:
    for msg in track:
        print(msg)

/home/zedroman/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/home/zedroman/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in less


32.12499999999992
22.124999999999982
40.37500000000002
42.75000000000005


KeyboardInterrupt: 

In [38]:
torch.save({
            'model_state_dict': decoder.state_dict(),
            'optimizer_state_dict': decoder_optimizer.state_dict(),
            'random_starts' : random_starts,
            'velocity_range' : velocity_range,
            'time_range' : time_range,
            'tempo' : tempo,
            'pedal_max' : pedal_max,
            'class_note' : class_note,
            }, '/home/zedroman/Documents/Sonia/models/anime2')
with open('/home/zedroman/Documents/Sonia/models/anime2_classes', 'w') as f: 
    f.write(str(classes))
    f.write(' ')
    f.write(str(hidden_size))

In [ ]:
decoder2 = DecoderRNN(hidden_size, classes, decoder_params).to(device)
decoder_optimizer2 = optim.Adam(decoder.parameters() , lr=learning_rate)
checkpoint = torch.load('/home/zedroman/Documents/Sonia/models/bethoven')
decoder2.load_state_dict(checkpoint['model_state_dict'])
random_starts2 = checkpoint['random_starts']
decoder_optimizer2.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
print(random_starts2==random_starts)
print(decoder2==decoder)
print(decoder_optimizer2==decoder_optimizer)

In [ ]:
generate(10, decoder2)

In [ ]:
with open('/home/zedroman/Documents/Sonia/models/anime_classes') as f: 
    data = int(f.readlines()[0])
    print(data)
    print(classes)

In [ ]:
# with open('/home/zedroman/Documents/Sonia/models/anime_classes', 'w') as f: 
#     f.write(str(classes))
#     f.write(' ')
#     f.write(str(hidden_size))

7
7
1
5
9
8
7
5
8
6
